## Cardinality

Các giá trị của một biến hạng mục được chọn từ một nhóm hạng mục còn được gọi là nhãn. Ví dụ: trong biến _gender_, các hạng mục hoặc nhãn là male và female trong khi ở biến _city_, các nhãn có thể là London, Manchester, Brighton,...

Các biến hạng mục khác nhau chứa lượng nhãn hoặc hạng mục khác nhau. Biến _gender_ chỉ chứa 2 nhãn, nhưng các biến như _city_ hoặc _postcode_ có thể chứa nhiều nhãn khác nhau.

Số lượng các nhãn khác nhau trong một biến hạng mục là cardinality. Số lượng nhiều các nhãn trong một biến được gọi là __high cardinality__.


### Nhiều nhãn trong một biến hạng mục có vấn đề không?

High cardinality có thể đối mặt với các vẫn đề sau:

- Các biến có quá nhiều nhãn có xu hướng chiếm ưu thế hơn so với những biến chỉ có một vài nhãn, đặc biệt là trong các thuật toán **cây**.

- Nhiều nhãn trong một biến có thể gây nhiễu với một ít thông tin (nếu có), do đó khiến các mô hình học máy dễ bị overfit.

- Một số nhãn có thể chỉ xuất hiện trong tập dữ liệu huấn luyện, không có trong tập kiểm tra, do đó các thuật toán học máy có thể quá khớp với tập huấn luyện.

- Ngược lại, một số nhãn có thể chỉ xuất hiện trong tập kiểm tra, do đó các thuật toán máy học không thể thực hiện phép tính với quan sát mới (không nhìn thấy).


Đặc biệt, **các phương thức cây có thể thiên về các biến có nhiều nhãn** (các biến với high cardinality). Do đó, chất lượng của chúng có thể bị ảnh hưởng bởi high cardinality.

Dưới đây, chúng ta sẽ trình bày tác động của biến có high cardinality với chất lượng của các thuật toán học máy khác nhau và cách khắc phục nhanh chóng để giảm số lượng nhãn mà không cần bất kỳ thông tin chi tiết dữ liệu nào để giúp tăng chất lượng.

## Trong bài lab này, chúng ta sẽ:

- Tìm hiểu cách định lượng cardinality
- Xem ví dụ về các biến có độ cardinality cao và thấp
- Hiểu được tác dụng của cardinality khi chuẩn bị các tập huấn luyện và tập kiểm tra
- Trực quan hóa tác động của cardinality đối với chất lượng của mô hình Học Máy

Chúng ta sẽ sử dụng tập dữ liệu Titanic.


In [23]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# xây dựng các mô hình học máy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# đánh giá mô hình
from sklearn.metrics import roc_auc_score

# tách thành tập huấn luyện và kiểm tra
from sklearn.model_selection import train_test_split

In [24]:
# load tập dữ liệu titanic 

data = pd.read_csv('dataset/titanic.csv')
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


Các hạng mục trong  tập dữ liệu này là Name, Sex, Ticket, Cabin và Embarked. 

---------------
**Lưu ý** rằng Ticket và Cabin chứa cả chữ cái và số nên chúng ta sẽ coi chúng là biến hỗn hợp. Trong mô phỏng này, hãy coi nó là biến hạng mục.

In [25]:
# hãy kiểm tra cardinality, đây là số
# nhãn khác nhau cho các biến hạng mục khác nhau

## Yêu cầu 1:
## VIẾT CODE Ở ĐÂY:
print('Number of categories in the variable Name: {}'.format(
    len(data.name.unique()))) 

print('Number of categories in the variable Gender: {}'.format(
    len(data.sex.unique())))

print('Number of categories in the variable Ticket: {}'.format(
    len(data.ticket.unique())))

print('Number of categories in the variable Cabin: {}'.format(
    len(data.cabin.unique())))

print('Number of categories in the variable Embarked: {}'.format(
    len(data.embarked.unique())))

print('Total number of passengers in the Titanic: {}'.format(len(data)))

Number of categories in the variable Name: 1307
Number of categories in the variable Gender: 2
Number of categories in the variable Ticket: 929
Number of categories in the variable Cabin: 182
Number of categories in the variable Embarked: 4
Total number of passengers in the Titanic: 1309


<details><summary> Gợi ý: </summary>

[unique()](https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html)

</details>

Trong khi biến Sex chỉ chứa 2 hạng mục và Embarked chứa 4 hạng mục (low cardinality) thì các biến Ticket, Name và Cabin chứa một số lượng lớn các nhãn khác nhau (high cardinality) như mong đợi.

Để chứng minh hiệu quả của high cardinality trong các tập huấn luyện và kiểm tra cũng như chất lượng học máy, chúng ta sẽ làm việc với biến Cabin. Tạo một biến mới với độ cardinality đã giảm. 

In [26]:
# hãy khám phá các giá trị/hạng mục của Cabin

# chúng ta biết rằng cell trước có 148
# cabin khác nhau, do đó biến cabin
# có độ cardinal cao

## Yêu cầu 2:
## VIẾT CODE Ở ĐÂY:
data.cabin.unique()

array(['B5', 'C22', 'E12', 'D7', 'A36', 'C101', nan, 'C62', 'B35', 'A23',
       'B58', 'D15', 'C6', 'D35', 'C148', 'C97', 'B49', 'C99', 'C52', 'T',
       'A31', 'C7', 'C103', 'D22', 'E33', 'A21', 'B10', 'B4', 'E40',
       'B38', 'E24', 'B51', 'B96', 'C46', 'E31', 'E8', 'B61', 'B77', 'A9',
       'C89', 'A14', 'E58', 'E49', 'E52', 'E45', 'B22', 'B26', 'C85',
       'E17', 'B71', 'B20', 'A34', 'C86', 'A16', 'A20', 'A18', 'C54',
       'C45', 'D20', 'A29', 'C95', 'E25', 'C111', 'C23', 'E36', 'D34',
       'D40', 'B39', 'B41', 'B102', 'C123', 'E63', 'C130', 'B86', 'C92',
       'A5', 'C51', 'B42', 'C91', 'C125', 'D10', 'B82', 'E50', 'D33',
       'C83', 'B94', 'D49', 'D45', 'B69', 'B11', 'E46', 'C39', 'B18',
       'D11', 'C93', 'B28', 'C49', 'B52', 'E60', 'C132', 'B37', 'D21',
       'D19', 'C124', 'D17', 'B101', 'D28', 'D6', 'D9', 'B80', 'C106',
       'B79', 'C47', 'D30', 'C90', 'E38', 'C78', 'C30', 'C118', 'D36',
       'D48', 'D47', 'C105', 'B36', 'B30', 'D43', 'B24', 'C2', 'C65',


Bây giờ hãy giảm cardinality của biến. Bằng cách nào? Thay vì sử dụng toàn bộ giá trị **cabin**, chúng ta sẽ chỉ lấy chữ cái đầu tiên.

***Cơ sở lý luận***: chữ cái đầu tiên cho biết tầng cabin, đó là dấu hiệu của cả địa vị tầng lớp xã hội và độ gần với bề mặt tàu Titanic. Cả hai đều có khả năng cải thiện xác suất sống sót. 

In [27]:
# hãy lấy chữ cái đầu tiên của Cabin

## Yêu cầu 3:
## VIẾT CODE Ở ĐÂY:
data['Cabin_reduced'] = data['cabin'].astype(str).str[0]

data[['cabin', 'Cabin_reduced']].head()

,cabin,Cabin_reduced
0,B5,B
1,C22,C
2,C22,C
3,C22,C
4,C22,C


<details><summary> Gợi ý: </summary>

[str()](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html)

</details>

In [28]:
print('Number of categories in the variable Cabin: {}'.format(
    len(data.cabin.unique())))

print('Number of categories in the variable Cabin reduced: {}'.format(
    len(data.Cabin_reduced.unique())))

Number of categories in the variable Cabin: 182
Number of categories in the variable Cabin reduced: 9


Chúng ta đã giảm số nhãn khác nhau từ 182 xuống còn 9.

In [29]:
from sklearn.model_selection import train_test_split

use_cols = ['cabin', 'Cabin_reduced', 'sex']

# hàm này từ scikit-learn
## Yêu cầu 4:
## VIẾT CODE Ở ĐÂY:
X_train, X_test, y_train, y_test = train_test_split(
    data[use_cols], 
    data['survived'],  
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((916, 3), (393, 3))

<details><summary> Gợi ý </summary>

[train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

</details>

### High cardinality dẫn tới phân phối không đều của các hạng mục trong tập huấn luyện và kiểm tra 

Khi một biến có độ cardinal cao, thường thì một số hạng mục chỉ nằm trong tập huấn luyện hoặc chỉ trong tập kiểm tra. Nếu chỉ xuất hiện trong tập huấn luyện, chúng có thể dẫn đến overfitting. Nếu chỉ xuất hiện trong tập kiểm tra thì thuật toán học máy sẽ không biết cách xử lý chúng vì nó không thấy chúng trong quá trình huấn luyện.

In [30]:
# Hãy tìm các nhãn chỉ xuất hiện trong tập huấn luyện

unique_to_train_set = [
    x for x in X_train.cabin.unique() if x not in X_test.cabin.unique()
]

len(unique_to_train_set)

113

Có 113 Cabin chỉ xuất hiện trong tập huấn luyện mà không có trong tập kiểm tra.

In [31]:
# Hãy tìm các nhãn chỉ xuất hiện trong tập kiểm tra

unique_to_test_set = [
    x for x in X_test.cabin.unique() if x not in X_train.cabin.unique()
]

len(unique_to_test_set)

36

Các biến với high cardinality có xu hướng có các giá trị (tức là hạng mục) xuất hiện trong tập huấn luyện mà không có trong tập kiểm tra và ngược lại. Điều này sẽ dẫn tới các vấn đề tại thời điểm huấn luyện (do overfitting) và tính điểm dữ liệu mới (mô hình nên xử lý như thế nào với các hạng mục không nhìn thấy được?).

Vấn đề này gần như được khắc phục bằng cách giảm cardinality của biến. Hãy xem bên dưới. 

In [32]:
# Hãy tìm các nhãn chỉ xuất hiện trong tập huấn luyện
# cho Cabin với cardinality đã giảm

## Yêu cầu 5:
## VIẾT CODE Ở ĐÂY:
unique_to_train_set = [
    x for x in X_train['Cabin_reduced'].unique()
    if x not in X_test['Cabin_reduced'].unique()
]

len(unique_to_train_set)

1

In [33]:
# Hãy tìm các nhãn chỉ xuất hiện trong tập kiểm tra
# cho Cabin với cardinality đã giảm

## Yêu cầu 6:
## VIẾT CODE Ở ĐÂY:
unique_to_test_set = [
    x for x in X_test['Cabin_reduced'].unique()
    if x not in X_train['Cabin_reduced'].unique()
]

len(unique_to_test_set)

0

Quan sát cách giảm cardinality, hiện chỉ có 1 nhãn trong tập huấn luyện mà không có trong tập kiểm tra. Không có nhãn nào trong tập kiểm tra mà không có trong tập huấn luyện.

### Tác động của cardinality tới chất lượng của mô hình Học Máy

Để đánh giá tác động của các biến hạng mục trong mô hình học máy, hãy nhanh chóng thay thế các hạng mục bằng số. Xem bên dưới. 

In [34]:
# Hãy ánh xạ lại Cabin thành các số để chúng ta có thể sử dụng nó nhằm huấn luyện mô hình Học máy

# thay thế từng cabin bằng một số
# để minh họa tác động của
# các nhãn trong thuật toán học máy

##############
# Lưu ý: đây không phải cách duy nhất, cũng không phải cách tốt nhất
# để mã hóa các biến hạng mục thành số
# còn có nhiều kỹ thuật khác trong phần
# "Encoding categorical variales"
##############

cabin_dict = {k: i for i, k in enumerate(X_train.cabin.unique(), 0)}
cabin_dict

{nan: 0,
 'E36': 1,
 'C68': 2,
 'E24': 3,
 'C22': 4,
 'D38': 5,
 'B50': 6,
 'A24': 7,
 'C111': 8,
 'F': 9,
 'C6': 10,
 'C87': 11,
 'E8': 12,
 'B45': 13,
 'C93': 14,
 'D28': 15,
 'D36': 16,
 'C125': 17,
 'B35': 18,
 'T': 19,
 'B73': 20,
 'B57': 21,
 'A26': 22,
 'A18': 23,
 'B96': 24,
 'G6': 25,
 'C78': 26,
 'C101': 27,
 'D9': 28,
 'D33': 29,
 'C128': 30,
 'E50': 31,
 'B26': 32,
 'B69': 33,
 'E121': 34,
 'C123': 35,
 'B94': 36,
 'A34': 37,
 'D': 38,
 'C39': 39,
 'D43': 40,
 'E31': 41,
 'B5': 42,
 'D17': 43,
 'F33': 44,
 'E44': 45,
 'D7': 46,
 'A21': 47,
 'D34': 48,
 'A29': 49,
 'D35': 50,
 'A11': 51,
 'B51': 52,
 'D46': 53,
 'E60': 54,
 'C30': 55,
 'D26': 56,
 'E68': 57,
 'A9': 58,
 'B71': 59,
 'D37': 60,
 'F2': 61,
 'C55': 62,
 'C89': 63,
 'C124': 64,
 'C23': 65,
 'C126': 66,
 'E49': 67,
 'E46': 68,
 'D19': 69,
 'B58': 70,
 'C82': 71,
 'B52': 72,
 'C92': 73,
 'E45': 74,
 'C65': 75,
 'E25': 76,
 'B3': 77,
 'D40': 78,
 'C91': 79,
 'B102': 80,
 'B61': 81,
 'A20': 82,
 'B36': 83,
 'C7': 84,

In [35]:
# thay các nhãn trong Cabin, sử dụng dic vừa tạo ở trên

## Yêu cầu 7:
## VIẾT CODE Ở ĐÂY:
X_train.loc[:, 'Cabin_mapped'] = X_train.loc[:, 'cabin'].map(cabin_dict)
X_test.loc[:, 'Cabin_mapped'] = X_test.loc[:, 'cabin'].map(cabin_dict)

X_train[['Cabin_mapped', 'cabin']].head(10)

,Cabin_mapped,cabin
501,0,NaN
588,0,NaN
402,0,NaN
1193,0,NaN
686,0,NaN
971,0,NaN
117,1,E36
540,0,NaN
294,2,C68
261,3,E24


<details><summary> Gợi ý </summary>

[map()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html)

</details>

Chúng ta thấy cách NaN nhận giá trị 0 trong biến mới, E36 nhận giá trị 1, C68 nhận giá trị 2,...

In [36]:
# Giờ chúng ta sẽ thay thế các chữ cái trong biến cabin đã giảm
# với quy trình tương tự

## Yêu cầu 8:
## VIẾT CODE Ở ĐÂY:

# tạo dictionary replace
cabin_dict = {k: i for i, k in enumerate(X_train['Cabin_reduced'].unique(), 0)}

# thay các nhãn bằng số với dictionary
X_train.loc[:, 'Cabin_reduced'] = X_train.loc[:, 'Cabin_reduced'].map(cabin_dict)
X_test.loc[:, 'Cabin_reduced'] = X_test.loc[:, 'Cabin_reduced'].map(cabin_dict)

X_train[['Cabin_reduced', 'cabin']].head(20)

,Cabin_reduced,cabin
501,0,NaN
588,0,NaN
402,0,NaN
1193,0,NaN
686,0,NaN
971,0,NaN
117,1,E36
540,0,NaN
294,2,C68
261,1,E24


Chúng ta thấy rằng E36 và E24 có cùng giá trị 1, vì chúng chỉ có chữ cái. Cả hai đều bắt đầu với E.

In [37]:
# ánh xạ lại biến hạng mục Sex thành số

X_train.loc[:, 'sex'] = X_train.loc[:, 'sex'].map({'male': 0, 'female': 1})
X_test.loc[:, 'sex'] = X_test.loc[:, 'sex'].map({'male': 0, 'female': 1})

X_train.sex.head()

501     1
588     1
402     1
1193    0
686     1
Name: sex, dtype: int64

In [38]:
# kiểm tra xem có giá trị bị khuyết nào trong các biến đó không
## Yêu cầu 9:
## VIẾT CODE Ở ĐÂY:
X_train[['Cabin_mapped', 'Cabin_reduced', 'sex']].isnull().sum()

Cabin_mapped     0
Cabin_reduced    0
sex              0
dtype: int64

<details><summary> Gợi ý </summary>

[isnull()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html)

</details>

In [39]:
## Yêu cầu 10:
## VIẾT CODE Ở ĐÂY:
X_test[['Cabin_mapped', 'Cabin_reduced', 'sex']].isnull().sum()

Cabin_mapped     41
Cabin_reduced     0
sex               0
dtype: int64

<details><summary> Gợi ý </summary>

[isnull()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isnull.html)

</details>

Trong tập kiểm tra, có 41 giá trị bị khuyết cho biến có độ cardinal cao. Chúng được giới thiệu khi mã hóa các hạng mục thành số.

Bằng cách nào? 

Nhiều hạng mục chỉ tồn tại trong tập kiểm tra. Do đó, khi tạo từ điển mã hóa chỉ với tập huấn luyện, chúng ta không tạo một số để thay thế các nhãn chỉ có trong tập kiểm tra. Do đó, chúng được mã hóa thành NaN. Chúng ta sẽ thấy cách giải quyết vấn đề này trong các notebook sau. Hiện tại, hãy điền các giá trị bị khuyết đó là 0. 

In [40]:
# hãy kiểm tra số hạng mục khác nhau trong các biến đã mã hóa
len(X_train.Cabin_mapped.unique()), len(X_train.Cabin_reduced.unique())

(147, 9)

Từ những điều trên, hãy lưu ý rằng từ 182 cabin ban đầu trong tập dữ liệu, chỉ có 147 cabin có mặt trong tập huấn luyện. Chúng ta cũng thấy cách giảm số lượng các hạng mục khác nhau xuống chỉ còn 9 hạng mục ở bước trước đó.

Hãy tiếp tục và đánh giá tác động của các nhãn trong các thuật toán học máy. 

### Random Forest (Rừng ngẫu nhiên)

In [41]:
# mô hình được tạo trong dữ liệu với high cardinality cho cabin

# gọi mô hình
rf = RandomForestClassifier(n_estimators=200, random_state=39)

# huấn luyện mô hình
rf.fit(X_train[['Cabin_mapped', 'sex']], y_train)

# dự đoán trên tập huấn luyện và kiểm tra
pred_train = rf.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = rf.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

## Yêu cầu 11:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Random Forests roc-auc: 0.853790650048556
Test set
Random Forests roc-auc: 0.7691361097284443


<details><summary> Gợi ý </summary>

[roc_auc_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)

</details>

Chúng ta quan sát thấy rằng chất lượng của Random Forest trong tập huấn tốt hơn nhiều so với trong tập kiểm tra. Điều này chỉ ra rằng mô hình bị overfitting, nghĩa là nó thực hiện rất tốt trong việc dự đoán kết quả trên tập dữ liệu mà nó đã được huấn luyện, nhưng thiếu khả năng tổng quát hóa dự đoán dữ liệu không nhìn thấy.

In [42]:
# mô hình đã xây trên dữ liệu với low cardinality cho cabin

# gọi mô hình
rf = RandomForestClassifier(n_estimators=200, random_state=39)

# huấn luyện mô hình
rf.fit(X_train[['Cabin_reduced', 'sex']], y_train)

# đưa ra dự đoán trên tập huấn luyện và tập kiểm tra
pred_train = rf.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = rf.predict_proba(X_test[['Cabin_reduced', 'sex']])

## Yêu cầu 12:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Random Forests roc-auc: 0.8163420365403872
Test set
Random Forests roc-auc: 0.8017670482827277


Bây giờ chúng ta thấy Random Forest không còn quá khớp với tập huấn luyện nữa. Ngoài ra, mô hình đã tổng quát hóa các dự đoán tốt hơn (so sánh roc-auc của mô hình này trên tập kiểm tra với roc-auc của mô hình trên cũng trong tập kiểm tra: 0.81 so với 0.80).

**Tôi muốn chỉ ra rằng, chúng ta có thể khắc phục ảnh hưởng của high cardinality bằng cách điều chỉnh các siêu tham số của Random Forest. Điều đó vượt ngoài phạm vi của khóa học này. Ở đây, tôi muốn chỉ ra rằng với mô hình tương tự, các siêu tham số giống hệt nhau thì high cardinality có thể khiến mô hình quá khớp.**

### AdaBoost

In [43]:
# mô hình xây trên dữ liệu với một vài hạng mục trong Cabin

# gọi mô hình
ada = AdaBoostClassifier(n_estimators=200, random_state=44)

# huấn luyện mô hình
ada.fit(X_train[['Cabin_mapped', 'sex']], y_train)

# dự đoán trên tập huấn luyện và kiểm tra
pred_train = ada.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = ada.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

print('Train set')
## Yêu cầu 12:
## VIẾT CODE Ở ĐÂY:
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Adaboost roc-auc: 0.8296861713101102
Test set
Adaboost roc-auc: 0.7604391350035948


In [44]:
# mô hình xây trên dữ liệu với một vài hạng mục trong biến Cabin

# gọi mô hình
ada = AdaBoostClassifier(n_estimators=200, random_state=44)

# huấn luyện mô hình
ada.fit(X_train[['Cabin_reduced', 'sex']], y_train)

# dự đoán trên tập huấn luyện và kiểm tra
pred_train = ada.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = ada.predict_proba(X_test[['Cabin_reduced', 'sex']].fillna(0))

## Yêu cầu 13:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Adaboost roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Adaboost roc-auc: 0.8161256723642566
Test set
Adaboost roc-auc: 0.8001078480172557


Tương tự, mô hình Adaboost được huấn luyện trong biến có high cardinality bị quá khớp với tập huấn luyện, trong khi Adaboost được huấn luyện trên biến có độ cardinal thấp không bị quá khớp, do đó thực hiện tổng quát hóa các dự đoán tốt hơn.

Ngoài ra, việc xây dựng AdaBoost trong mô hình có ít hạng mục hơn trong Cabin, a) đơn giản hơn và b) nếu một hạng mục khác xuất hiện trong tập kiểm tra, chỉ cần lấy chữ cái đầu của cabin, mô hình ML sẽ biết cách xử lý nó bởi vì nó đã thấy trong quá trình huấn luyện. 

### Hồi quy Logistic

In [45]:
# mô hình xây dựng trên dữ liệu với nhiều hạng mục trong biến Cabin

# gọi mô hình
logit = LogisticRegression(random_state=44, solver='lbfgs')

# huấn luyện mô hình
logit.fit(X_train[['Cabin_mapped', 'sex']], y_train)

# dự đoán trên tập huấn luyện và tập kiểm tra
pred_train = logit.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = logit.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

## Yêu cầu 14:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Logistic regression roc-auc: 0.8133909298124677
Test set
Logistic regression roc-auc: 0.7750815773463858


In [46]:
# mô hình xây dựng trên dữ liệu với ít hạng mục hơn trong biến Cabin

# gọi mô hình
logit = LogisticRegression(random_state=44, solver='lbfgs')

# huấn luyện mô hình
logit.fit(X_train[['Cabin_reduced', 'sex']], y_train)

# dự đoán trên tập huấn luyện và tập kiểm tra
pred_train = logit.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = logit.predict_proba(X_test[['Cabin_reduced', 'sex']].fillna(0))

## Yêu cầu 15:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Logistic regression roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Logistic regression roc-auc: 0.8123468468695123
Test set
Logistic regression roc-auc: 0.8008268347989602


Có thể rút ra kết luận tương tự cho Hồi quy Logistic: giảm cardinality giúp cải thiện chất lượng và tính tổng quát của thuật toán.

### Gradient Boosted Classifier

In [47]:
# mô hình xây dựng trên dữ liệu với nhiều hạng mục trong biến Cabin

# gọi mô hình
gbc = GradientBoostingClassifier(n_estimators=300, random_state=44)

# huấn luyện mô hình
gbc.fit(X_train[['Cabin_mapped', 'sex']], y_train)

# dự đoán trên tập huấn luyện và tập kiểm tra
pred_train = gbc.predict_proba(X_train[['Cabin_mapped', 'sex']])
pred_test = gbc.predict_proba(X_test[['Cabin_mapped', 'sex']].fillna(0))

## Yêu cầu 16:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Gradient Boosted Trees roc-auc: 0.862631390919749
Test set
Gradient Boosted Trees roc-auc: 0.7733117637298823


In [48]:
# mô hình xây dựng trên dữ liệu với nhiều hạng mục trong biến Cabin

# gọi mô hình
gbc = GradientBoostingClassifier(n_estimators=300, random_state=44)

# huấn luyện mô hình
gbc.fit(X_train[['Cabin_reduced', 'sex']], y_train)

# dự đoán trên tập huấn luyện và tập kiểm tra
pred_train = gbc.predict_proba(X_train[['Cabin_reduced', 'sex']])
pred_test = gbc.predict_proba(X_test[['Cabin_reduced', 'sex']].fillna(0))

## Yêu cầu 17:
## VIẾT CODE Ở ĐÂY:
print('Train set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_train, pred_train[:,1])))
print('Test set')
print('Gradient Boosted Trees roc-auc: {}'.format(roc_auc_score(y_test, pred_test[:,1])))

Train set
Gradient Boosted Trees roc-auc: 0.816719415917359
Test set
Gradient Boosted Trees roc-auc: 0.8015181682429069


Các cây Gradient Boosted thực sự quá khớp với tập huấn luyện trong trường hợp biến Cabin có nhiều nhãn. Điều này đã được dự tính vì các phương pháp cây có xu hướng thiên về các biến có nhiều hạng mục.

